In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

# Test functions

In [ ]:
#export
def dice():
    return 6

There is some explanation here, and an example:

In [ ]:
dice()

6

In [ ]:
# assert say_hello("Jeremy")=="Hello Jeremy!" # Test function